# 🧭 AI-Risk Radar: Monitoring Global AI Developments with LLMs & Clustering

This interactive notebook tracks emerging AI-related risks and policy trends by:

- Scraping real-time news from trusted technology and policy sources  
- Embedding and clustering articles using semantic similarity  
- Labeling topics with OpenAI's `gpt-4o-mini` model  
- Generating concise cluster summaries via LLMs  
- Presenting everything in an intuitive Gradio dashboard

Use this tool to explore how AI is shaping geopolitics, energy infrastructure, ethics, and regulation — all in one place.


### 📦 Install Required Libraries for AI Risk Radar Pipeline


In [ ]:
!pip install openai newspaper3k feedparser \
             sentence-transformers chromadb umap-learn hdbscan \
             plotly dash langchain-community lxml-html-clean gradio

### 🧠 Import Standard Libraries and External Modules


In [ ]:
# Standard library imports
import os, datetime, json, pathlib, textwrap, tempfile, itertools
from typing import List, Dict, Any  # For type annotations

# News feed parsing and article extraction
import feedparser         # Parse RSS/Atom feeds
import newspaper          # Scrape full-text news articles

# Data manipulation and progress bars
import pandas as pd
from tqdm.auto import tqdm  # Smart progress bar for loops

# Embeddings and vector database
from sentence_transformers import SentenceTransformer  # To embed article text
import chromadb                                          # Vector DB for similarity search
from chromadb.utils import embedding_functions           # (Optional utility functions)

# Dimensionality reduction and clustering
import umap     # For 2D projection of embeddings
import hdbscan  # Hierarchical density-based clustering
from sklearn.cluster import KMeans # Clustering method

# OpenAI API for summaries and labeling
import openai

# Gradio UI framework for building the interactive app
import gradio as gr

# Dashboard components (was used for Dash-based UI)
import plotly.express as px
from dash import Dash, html, dcc, dash_table, Input, Output

# 🌐 Google Colab utility for securely loading API keys
from google.colab import userdata

### ⚙️ Runtime Configuration and API Setup


In [ ]:
# Runtime settings
BATCH_SIZE = 8  # Number of articles to process in one embedding batch
EMBED_MODEL_NAME = "intfloat/e5-large-v2"  # SentenceTransformer model for multilingual embeddings
VECTOR_DB_DIR = "./chroma_db"  # Directory for persistent Chroma vector database

# News feeds to monitor
RSS_FEEDS = [
    "https://www.euronews.com/tag/artificial-intelligence/rss",  # Euronews AI section
    "https://www.brookings.edu/topic/artificial-intelligence/feed/",  # Brookings Institution
    "https://news.un.org/feed/subscribe/en/news/topic/technology/feed/rss.xml",  # UN tech news
    "https://www.technologyreview.com/feed/",  # MIT Tech Review
    "https://feeds.arstechnica.com/arstechnica/technology-lab",  # Ars Technica tech lab
]

# OpenAI API setup via Google Colab secure storage
openai.api_key = userdata.get("OPENAI_API_KEY")  # Securely load OpenAI key from Colab userdata
if openai.api_key is None:
    raise ValueError("❌ OPENAI_API_KEY not found in userdata. Please set it using: userdata.set('OPENAI_API_KEY', 'sk-...')")
print("✅ OpenAI API key loaded successfully.")

✅ OpenAI API key loaded successfully.


### 📡 AI-Risk Radar: Multi-Agent Pipeline Definition

This cell defines all core components of the AI-Risk Radar system. It includes modular agents that:

- 🔍 **Scrape** news from various AI-related RSS feeds  
- 🧠 **Embed** articles using sentence-transformers (stored in ChromaDB for persistence)  
- 🧭 **Cluster** similar articles using UMAP + HDBSCAN or KMeans  
- 🏷️ **Label** each cluster with a concise topic using an LLM (`gpt-4o-mini`)  
- 📝 **Summarize** each cluster’s key themes via OpenAI’s GPT API  
- 💻 **Visualize** everything in a Gradio-powered interactive dashboard

The system provides a high-level overview of AI-related global developments, with options to download per-cluster data or generate a weekly summary using LLM analysis.


In [ ]:
class NewsScraperAgent:
    """Fetch articles from RSS feeds and build a cleaned DataFrame."""

    def __init__(self, feeds: List[str]):
        self.feeds = feeds

    def run(self) -> pd.DataFrame:
        records = []
        for url in self.feeds:
            print(f"📡 Parsing feed: {url}")
            fp = feedparser.parse(url)  # Parse RSS feed
            print(f"  → Found {len(fp.entries)} entries")
            for entry in fp.entries:
                records.append({
                    "title": entry.get("title", ""),
                    "link": entry.get("link", ""),
                    "published": entry.get("published", ""),
                    "summary": entry.get("summary", ""),
                })

        df = pd.DataFrame(records)

        if df.empty:
            print("⚠️ No entries found in feeds.")
            return df

        # Download full articles using newspaper3k
        texts = []
        for link in tqdm(df["link"], desc="📥 Downloading articles"):
            try:
                a = newspaper.Article(link, language="en")
                a.download()
                a.parse()
                texts.append(a.text)
            except Exception:
                texts.append("")

        df["text"] = texts
        df["published_dt"] = pd.to_datetime(df.published, errors="coerce")

        # Filter: remove short, empty, or paywalled articles
        def is_valid_article(text):
            text = text.lower().strip()
            return (
                len(text) > 300 and
                "try unlimited access" not in text and
                "subscribe" not in text and
                "join ft edit" not in text
            )

        df = df[df["text"].apply(is_valid_article)].reset_index(drop=True)
        print(f"✅ Retained {len(df)} valid articles after filtering")

        return df


class EmbeddingAgent:
    """Embed articles using SentenceTransformer and store them in Chroma vector DB."""

    def __init__(self, model_name: str, persist_dir: str):
        from sentence_transformers import SentenceTransformer
        import chromadb

        self.embedder = SentenceTransformer(model_name)  # Load embedding model
        self.client = chromadb.PersistentClient(path=persist_dir)  # Set up persistent DB

        try:
            self.client.delete_collection("articles")  # Reset collection if it exists
            print("🧼 Deleted existing 'articles' collection to reset it")
        except Exception as e:
            print("⚠️ Could not delete existing collection:", e)

        self.collection = self.client.get_or_create_collection("articles")

    def run(self, df: pd.DataFrame) -> pd.DataFrame:
        texts = []
        ids = []
        metadatas = []

        for i, row in df.iterrows():
            if not row.text or not row.text.strip():
                continue
            metadata = {
                k: (str(v) if isinstance(v, (pd.Timestamp, datetime.datetime)) else v)
                for k, v in row.to_dict().items()
            }
            texts.append(row.text)
            ids.append(str(i))
            metadatas.append(metadata)

        if not texts:
            raise ValueError("❌ No valid texts found for embedding.")

        print(f"📄 Generating embeddings for {len(texts)} documents...")
        vectors = self.embedder.encode(texts, show_progress_bar=True).tolist()

        self.collection.add(
            ids=ids,
            documents=texts,
            metadatas=metadatas,
            embeddings=vectors
        )

        print(f"✅ Added {len(vectors)} embeddings to ChromaDB")

        return pd.DataFrame(vectors)


class ClusteringAgent:
    """Cluster and label articles using unsupervised ML and LLM-based labeling."""

    def __init__(self):
        pass  # Could configure LLM settings here if needed

    def label_cluster_with_llm(self, texts: List[str], model="gpt-4o-mini") -> str:
        # Use OpenAI LLM to assign a single label for the cluster
        try:
            joined = "\n\n".join(texts)[:8000]
            response = openai.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are an expert in analyzing clusters of geopolitical news."},
                    {"role": "user", "content": (
                        "You are an expert policy analyst. "
                        "Given the following articles in a single topic cluster, return ONLY ONE short, clear, and descriptive topic label "
                        "(5 words or fewer). Do NOT include bullet points or markdown. Just return the label text.\n\n"
                        f"{joined}"
                    )}
                ],
                temperature=0.2,
            )
            label = response.choices[0].message.content.strip()
            return label
        except Exception as e:
            print(f"⚠️ LLM labelling failed: {e}")
            return "Unlabelled"

    def run(self, embeddings_df: pd.DataFrame, df: pd.DataFrame) -> tuple:
        import umap
        import hdbscan

        # Reduce dimensionality for clustering and plotting
        reducer = umap.UMAP(n_neighbors=5, min_dist=0.0, metric="cosine")
        embedding_2d = reducer.fit_transform(embeddings_df.values)

        # Apply HDBSCAN clustering
        clusterer = hdbscan.HDBSCAN(min_cluster_size=2, metric="euclidean")
        labels = clusterer.fit_predict(embedding_2d)

        if len(set(labels)) <= 1 or all(label == -1 for label in labels):
            print("⚠️ HDBSCAN found no valid clusters — falling back to KMeans.")
            kmeans = KMeans(n_clusters=3, random_state=42)
            labels = kmeans.fit_predict(embedding_2d)

        df_clustered = df.copy()
        df_clustered["cluster"] = labels

        # Label each cluster using LLM
        cluster_labels = {}
        for c in set(labels):
            texts = df_clustered[df_clustered.cluster == c].text.tolist()
            label = self.label_cluster_with_llm(texts)
            cluster_labels[c] = label

        df_clustered["topic"] = df_clustered["cluster"].map(cluster_labels)

        return df_clustered, embedding_2d, cluster_labels


class SummariserAgent:
    """Summarise each cluster into a brief using OpenAI."""

    def __init__(self, model="gpt-3.5-turbo"):
        import openai
        from google.colab import userdata

        self.openai = openai
        self.model = model
        self.api_key = userdata.get("OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("❌ OPENAI_API_KEY not set in Colab. Use `userdata.set(...)` to define it.")
        self.openai.api_key = self.api_key

    def summarise_with_openai(self, text: str) -> str:
        # Call OpenAI to summarize the joined article texts
        try:
            response = self.openai.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "You are an expert geopolitical analyst."},
                    {"role": "user", "content": f"Summarise the following cluster of news in 150 words:\n\n{text}"}
                ],
                temperature=0.3,
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            return f"(OpenAI error: {str(e)})"

    def run(self, df_clustered: pd.DataFrame) -> Dict[int, str]:
        summaries = {}
        for c in df_clustered.cluster.unique():
            texts = df_clustered[df_clustered.cluster == c].text.tolist()
            joined = "\n".join(texts)[:6000]  # Truncate input
            summary = self.summarise_with_openai(joined)
            summaries[c] = summary
        return summaries


class GradioDashboardAgent:
    """Interactive Gradio dashboard for exploring article clusters."""

    def __init__(self, df_clustered, embedding_2d, summaries, model="gpt-4o-mini"):
        self.df = df_clustered.copy()
        self.df["x"] = embedding_2d[:, 0]
        self.df["y"] = embedding_2d[:, 1]
        self.summaries = summaries

        self.openai = openai
        self.openai.api_key = openai.api_key
        self.model = model

    def generate_weekly_summary_txt(self):
        # Generate weekly summary from recent articles using LLM
        now = pd.Timestamp.utcnow()
        week_ago = now - pd.Timedelta(days=7)

        published_dates = pd.to_datetime(self.df["published"], errors="coerce", utc=True)
        recent_articles = self.df[published_dates >= week_ago]["text"].dropna().tolist()

        if not recent_articles:
            summary = "No news articles were published in the last 7 days."
        else:
            joined_text = "\n\n".join(recent_articles)[:8000]

            try:
                response = self.openai.chat.completions.create(
                    model=self.model,
                    messages=[
                        {"role": "system", "content": "You are an expert geopolitical analyst."},
                        {"role": "user", "content": f"Write a policy summary based on this AI news coverage:\n\n{joined_text}"}
                    ],
                    temperature=0.3,
                )
                summary = response.choices[0].message.content.strip()
            except Exception as e:
                summary = f"(OpenAI error: {str(e)})"

        tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".txt", mode="w", encoding="utf-8")
        tmp.write(summary)
        tmp.close()
        return tmp.name

    def launch(self, share=False):
        df_plot = self.df

        fig = px.scatter(
            df_plot,
            x="x",
            y="y",
            color="topic",
            hover_name="title",
            hover_data={"x": False, "y": False, "published": True, "link": False}
        )

        def get_cluster_info(cluster_id):
            # Display summary and article list for a selected cluster
            cluster_id = int(cluster_id)
            summary = self.summaries.get(cluster_id, "(No summary)")
            table = df_plot[df_plot["cluster"] == cluster_id][["title", "published", "link"]].copy()
            table.columns = ["Title", "Published", "Link"]
            return summary, table

        def download_csv(cluster_id):
            # Download articles in selected cluster as CSV
            cluster_id = int(cluster_id)
            table = df_plot[df_plot["cluster"] == cluster_id][["title", "published", "link"]].copy()
            table.columns = ["Title", "Published", "Link"]
            tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".csv")
            table.to_csv(tmp_file.name, index=False)
            return tmp_file.name

        cluster_options = df_plot[["cluster", "topic"]].drop_duplicates().sort_values("topic")
        choices = [(row["topic"], row["cluster"]) for _, row in cluster_options.iterrows()]

        with gr.Blocks() as app:
            with gr.Row():
                gr.Markdown("<h2 style='text-align: center; font-weight: bold;'>🧭 AI-Risk Radar — Cluster Explorer</h2>")

            with gr.Row():
                plot = gr.Plot(value=fig, label="Cluster Map")

            with gr.Row():
                dropdown = gr.Dropdown(choices=choices, label="🎯 Select Topic", interactive=True)

            with gr.Row():
                button = gr.Button("🔍 Show Cluster Info")

            summary = gr.Textbox(label="📝 Cluster Summary", lines=5, interactive=False)
            table = gr.DataFrame(label="📰 Articles", interactive=False)

            with gr.Row():
                download_btn = gr.Button("🗞️ Download News Articles (CSV)")
                weekly_report_btn = gr.Button("📅 Generate Weekly Report (.txt)")

            with gr.Row():
                download_file = gr.File(label="📄 Download Link")
                weekly_report_file = gr.File(label="🗂️ Weekly Report Download")

            # Bind dashboard events
            button.click(fn=get_cluster_info, inputs=dropdown, outputs=[summary, table])
            download_btn.click(fn=download_csv, inputs=dropdown, outputs=download_file)
            weekly_report_btn.click(fn=self.generate_weekly_summary_txt, inputs=[], outputs=weekly_report_file)

        app.launch(share=share, debug=True)

### 🚀 Run AI-Risk Radar Pipeline

This cell runs the full pipeline:

- 📰 Scrape news from RSS feeds  
- 🧠 Embed articles (saved in ChromaDB)  
- 🧭 Cluster and label topics with LLM  
- 📝 Summarize clusters via OpenAI  
- 💻 Launch interactive Gradio dashboard


In [ ]:
print("🏃‍ Starting multi-agent pipeline…")

# Step 1: Scrape news articles from RSS feeds
news_agent = NewsScraperAgent(RSS_FEEDS)
df_articles = news_agent.run()
print(f"📰 Fetched {len(df_articles)} articles")
# Stop if no valid articles were scraped
if df_articles.empty:
    raise RuntimeError("❌ No valid articles were scraped. Exiting.")

# Step 2: Embed articles into vector representations and store in ChromaDB
embed_agent = EmbeddingAgent(EMBED_MODEL_NAME, VECTOR_DB_DIR)
embeddings_df = embed_agent.run(df_articles)
# Stop if embeddings failed
if embeddings_df.empty or embeddings_df.shape[0] == 0:
    raise RuntimeError("❌ Embedding failed. Check input text and embedding process.")
print("✅ Embedding complete")

# Step 3: Cluster embedded articles and label each cluster using LLM
cluster_agent = ClusteringAgent()
df_clustered, emb2d, cluster_labels = cluster_agent.run(embeddings_df, df_articles)
print("🏷️ Cluster labels:", cluster_labels)

# Step 4: Summarize each cluster using OpenAI (LLM)
summariser = SummariserAgent(model="gpt-4o-mini")  # You can switch to "gpt-3.5-turbo" if needed
cluster_summaries = summariser.run(df_clustered)
print("📝 Summarisation complete")

# Step 5: Launch the interactive Gradio dashboard
gradio_agent = GradioDashboardAgent(df_clustered, emb2d, cluster_summaries)
print("🚀 Launching Gradio app...")
gradio_agent.launch(share=True)  # Set to False to run locally without public link